In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
data_path = '../data/'
df = pd.read_csv(os.path.join(data_path, 'WA_Fn-UseC_-HR-Employee-Attrition.csv'))
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


There are 35 features in total in the dataset about. 
Let's focus on a few of them:
- Age
- Attrition
- Department
- DistanceFromHome
- Education
- EnvironmentSatisfaction
- Gender
- JobSatisfaction
- MaritalStatus
- MonthlyIncome
- OverTime
- PerformanceRating
- RelationshipSatisfaction
- TotalWorkingYears
- YearsAtCompany

In this lab, we will use attrition as our label, to try to predict the attrition status accroding to other attributes. 


In [4]:
to_keep = {'Age', 'Attrition', 'Department','DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'Gender', 'JobSatisfaction', 'MaritalStatus',
           'MonthlyIncome', 'OverTime', 'PerformanceRating', 'RelationshipSatisfaction','TotalWorkingYears','YearsAtCompany'}
to_drop = set(df.columns)-to_keep
df.drop(to_drop, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 15 columns):
Age                         1470 non-null int64
Attrition                   1470 non-null object
Department                  1470 non-null object
DistanceFromHome            1470 non-null int64
Education                   1470 non-null int64
EnvironmentSatisfaction     1470 non-null int64
Gender                      1470 non-null object
JobSatisfaction             1470 non-null int64
MaritalStatus               1470 non-null object
MonthlyIncome               1470 non-null int64
OverTime                    1470 non-null object
PerformanceRating           1470 non-null int64
RelationshipSatisfaction    1470 non-null int64
TotalWorkingYears           1470 non-null int64
YearsAtCompany              1470 non-null int64
dtypes: int64(10), object(5)
memory usage: 172.3+ KB


# Preprocessing

It's good that we don't have any null value. Let's one hot encode the Attrition, Department, Gender, MaritalStatus and Overtime. 

In [5]:
to_convert = ['Education','EnvironmentSatisfaction','JobSatisfaction',
            'PerformanceRating','RelationshipSatisfaction']
for col in to_convert:
    df[col] = df[col].astype(np.str)
    

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

to_encode = {'Attrition', 'Department','Gender','MaritalStatus','OverTime','Education','EnvironmentSatisfaction','JobSatisfaction',
            'PerformanceRating','RelationshipSatisfaction'}
encoders = dict()

for col in to_encode:
    if col=="attrition":
        tmp = LabelEncoder()
        df[col] = tmp.fit_transform(df[col])
    else:
        encoders[col] = LabelEncoder()
        df[col+'_int'] = encoders[col].fit_transform(df[col])
    

Then, let's scale the numeric features. 

In [7]:
categorical_features =list(to_encode)
categorical_features = [x+'_int' for x in categorical_features]
numerics = set(df.columns) - to_encode
numerics = list(numerics - set(categorical_features))

for atr in numerics:
    df[atr] = df[atr].astype(np.float)    
    ss = StandardScaler()
    df[atr] = ss.fit_transform(df[atr].values.reshape(-1, 1))
    
feature_columns = categorical_features + numerics

In [8]:
numerics

['DistanceFromHome',
 'MonthlyIncome',
 'YearsAtCompany',
 'TotalWorkingYears',
 'Age']

In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 25 columns):
Age                             1470 non-null float64
Attrition                       1470 non-null object
Department                      1470 non-null object
DistanceFromHome                1470 non-null float64
Education                       1470 non-null object
EnvironmentSatisfaction         1470 non-null object
Gender                          1470 non-null object
JobSatisfaction                 1470 non-null object
MaritalStatus                   1470 non-null object
MonthlyIncome                   1470 non-null float64
OverTime                        1470 non-null object
PerformanceRating               1470 non-null object
RelationshipSatisfaction        1470 non-null object
TotalWorkingYears               1470 non-null float64
YearsAtCompany                  1470 non-null float64
PerformanceRating_int           1470 non-null int64
MaritalStatus_int               1470 non-

In [19]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics as mt

# stratified 90/10 train/test split`
df_train, df_test = train_test_split(df, test_size=0.1, stratify=df.Attrition)

X_train = ss.fit_transform(df_train[feature_columns].values).astype(np.float32)
X_test = ss.fit_transform(df_test[feature_columns].values).astype(np.float32)

y_train = df_train['Attrition_int'].values.astype(np.int)
y_test = df_test['Attrition_int'].values.astype(np.int)

print('train', X_train.shape, 'test', X_test.shape)

train (1323, 15) test (147, 15)


In [20]:
cross_columns = [['Gender','MaritalStatus'],
                    ['Education', 'JobSatisfaction'],['Department','PerformanceRating'],
                    ['Education', 'JobSatisfaction','RelationshipSatisfaction'],['Department','OverTime'],
                ]

In [21]:
ohe = OneHotEncoder()
X_train_ohe = ohe.fit_transform(df_train[categorical_features].values)
X_test_ohe = ohe.transform(df_test[categorical_features].values)


X_train_num =  df_train[numerics].values
X_test_num = df_test[numerics].values

In [22]:

# we need to create separate sequential models for each embedding
embed_branches = []
X_ints_train = []
X_ints_test = []
all_inputs = []
all_branch_outputs = []

for cols in cross_columns:
    # encode crossed columns as ints for the embedding
    enc = LabelEncoder()
    
    # create crossed labels
    # needs to be commented better, Eric!
    X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
    X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)
    
    enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
    X_crossed_train = enc.transform(X_crossed_train)
    X_crossed_test = enc.transform(X_crossed_test)
    X_ints_train.append( X_crossed_train )
    X_ints_test.append( X_crossed_test )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32', name = '_'.join(cols))
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)
    
# merge the branches together
wide_branch = concatenate(all_branch_outputs)

# reset this input branch
all_branch_outputs = []
# add in the embeddings
for col in categorical_features:
    # encode as ints for the embedding
    X_ints_train.append( df_train[col].values )
    X_ints_test.append( df_test[col].values )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32', name=col)
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)
    
# also get a dense branch of the numeric features
all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
x = Dense(units=20, activation='relu')(all_inputs[-1])
all_branch_outputs.append( x )

# merge the branches together
deep_branch = concatenate(all_branch_outputs)
deep_branch = Dense(units=50,activation='relu')(deep_branch)
deep_branch = Dense(units=10,activation='relu')(deep_branch)
    
final_branch = concatenate([wide_branch, deep_branch])
final_branch = Dense(units=1,activation='sigmoid')(final_branch)

model = Model(inputs=all_inputs, outputs=final_branch)

model.compile(optimizer='adagrad',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit(X_ints_train+ [X_train_num],
        y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
1323/1323 [==============================] - 1s 1ms/step - loss: 0.1550 - acc: 0.8231
Epoch 2/10
1323/1323 [==============================] - 0s 114us/step - loss: 0.1145 - acc: 0.8405
Epoch 3/10
1323/1323 [==============================] - 0s 116us/step - loss: 0.0658 - acc: 0.9229
Epoch 4/10
1323/1323 [==============================] - 0s 87us/step - loss: 0.0178 - acc: 0.9970
Epoch 5/10
1323/1323 [==============================] - 0s 138us/step - loss: 0.0056 - acc: 1.0000
Epoch 6/10
1323/1323 [==============================] - 0s 117us/step - loss: 0.0028 - acc: 1.0000
Epoch 7/10
1323/1323 [==============================] - 0s 94us/step - loss: 0.0017 - acc: 1.0000
Epoch 8/10
1323/1323 [==============================] - 0s 143us/step - loss: 0.0012 - acc: 1.0000
Epoch 9/10
1323/1323 [==============================] - 0s 149us/step - loss: 8.6133e-04 - acc: 1.0000
Epoch 10/10
1323/1323 [==============================] - 0s 122us/step - loss: 6.6132e-04 - acc: 1.0000


In [23]:
yhat = np.round(model.predict(X_ints_test + [X_test_num]))
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

[[123   0]
 [  0  24]] 1.0
